In [1]:
import Pkg

package_list = ["DataFrames", "CSV",
"JuMP", "Gurobi",
 "LinearAlgebra", "Random", "Printf", "StatsBase", "CategoricalArrays",
  "Plots", "StatsPlots",
  "Distributions", "JLD"]

for pkg_name in package_list
    Pkg.add(pkg_name)
end

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Ch

In [2]:
using DataFrames, CSV
using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions

In [3]:
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


Gurobi.Env(Ptr{Nothing} @0x00007faf3e832e00, false, 0)

In [512]:
# read in csv file
v = CSV.read("/Users/zachwayne/Documents/GitHub/15093-project/volatility.csv", DataFrame)
r = CSV.read("/Users/zachwayne/Documents/GitHub/15093-project/returns.csv", DataFrame)
s = CSV.read("/Users/zachwayne/Documents/GitHub/15093-project/sectors.csv", DataFrame)
p = CSV.read("/Users/zachwayne/Documents/GitHub/15093-project/price.csv", DataFrame)
cor = CSV.read("/Users/zachwayne/Documents/GitHub/15093-project/correlations.csv", DataFrame)
cov = CSV.read("/Users/zachwayne/Documents/GitHub/15093-project/covariance.csv", DataFrame)
tbill = 0.029339583


0.029339583

In [513]:
# create new dataframe 
df = DataFrame(Stock = names(p)[2:end])
# column names of df are the column names of s
for i=2:12
    df[!, names(s)[i]] = s[:,i]
end
# loop through sector and add row to df if stock names match
for i=1:stocks
    #loop through column 1 of s
    for j=1:50
        if s[j,1] == names(p)[i+1]
            for k=2:12
                df[i, names(s)[k]] = s[j,k]             
            end
        end
    end
    
end

sec=df

,Stock,Communication Services,Consumer Discretionary,Consumer Staples,Energy
,String,Int64,Int64,Int64,Int64
1,NTRS,0,0,0,0
2,AVY,0,0,0,0
3,AFL,0,0,0,0
4,REGN,0,0,0,0
5,DAL,0,0,0,0
6,COO,0,0,0,0
7,CME,0,0,0,0
8,BK,0,0,0,0
9,O,0,0,0,0


In [534]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV));
stocks=50
k = 10

10

In [535]:
@variable(model, x[1:stocks] >= 0, Int)
@variable(model, z[1:stocks], Bin);
#@variable(model, 0<=w[1:stocks]<=1);
@variable(model, y[1:stocks]>=0);


In [536]:
@constraint(model, linking[i=1:stocks], y[i] <= 1000000*z[i]);  # linking 'bought stock' to 'how many we buy'
@constraint(model, linking2[i=1:stocks], y[i] >= z[i]);
@constraint(model, linking3[i=1:stocks], sum((y'.*p[:,2:end])[1,i] for i in 1:stocks) <= 300000*sum(y[i] for i=1:stocks));
@constraint(model, sum(z[i] for i=1:stocks) == k); 
@constraint(model,[j=2:12],sum(z[i]*sec[i,j] for i=1:stocks) <= 2);

# for all stocks we purchase, their c[i,j] must be <= .3
for i=1:stocks
    for j=1:stocks
        if i != j
            @constraint(model, z[i]*z[j]*cor[i,j+1] <= .3)
            @constraint(model, z[i]*z[j]*cor[i,j+1] >= -.3)
        end
    end
end
@constraint(model, r[:, 2]'*y == 1);

In [537]:
#@objective(model, Max, (sum(w[i] * r[i,2] for i=1:stocks)-tbill)/(sum((w[i]^2) * (v[i,2]^2) for i=1:stocks)+2*(sum(w[i]*w[j]*cov[i,j+1] for i=1:stocks, j=i+1:stocks))));
#@objective(model, Max, (sum((y[i]^2) * (v[i,2]^2) for i=1:stocks)+2*(sum(y[i]cov[i,j+1]*y[j] for i=1:stocks, j=i+1:stocks))));
# Set NonConvex parameter to 2 to solve model.
@objective(model, Min, y'*Matrix(cov[:, 2:end])*y);

In [538]:
optimize!(model);

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 163 rows, 150 columns and 2850 nonzeros
Model fingerprint: 0x26dd242f
Model has 1275 quadratic objective terms
Model has 4900 quadratic constraints
Variable types: 0 continuous, 150 integer (50 binary)
Coefficient statistics:
  Matrix range     [5e-04, 1e+06]
  QMatrix range    [1e-03, 9e-01]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e-05, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [3e-01, 3e-01]
Presolve removed 53 rows and 50 columns
Presolve time: 0.01s
Presolved: 1286 rows, 688 columns, 2697 nonzeros
Presolved model has 1275 quadratic objective terms
Variable types: 0 continuous, 688 integer (638 binary)

Root relaxation: objective 5.785211e+00, 339 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |    

In [539]:
z_opt = value.(z)
y_opt = value.(y)
#

50-element Vector{Float64}:
 -0.0
 26.0
 -0.0
 -0.0
 -0.0
 13.0
 -0.0
 -0.0
 -0.0
  1.0
 -0.0
 -0.0
  1.0
  ⋮
 -0.0
 -0.0
 -0.0
 41.0
 31.0
 -0.0
 -0.0
 -0.0
  0.0
 -0.0
 -0.0
 -0.0

In [540]:
# loop through z and print out the stocks we buy
for i=1:stocks
    if z_opt[i] > 0.1
        z_opt[i] = 1
    else
        z_opt[i] = 0
    end

end 

In [541]:
# xi = yi/sum(yi)
# loop through y
for i=1:stocks
    y_opt[i] = y_opt[i]*z_opt[i]
end
# loop through stocks and caluclate xi
x = zeros(stocks)
for i=1:stocks
    x[i] = y_opt[i]/sum(y_opt)
end

In [542]:
# loop thorugh x and show which stocks we buy
for i=1:stocks
    if x[i] > 0
        println("Stock ", names(p)[i+1], " is ", x[i]*100, "% of portfolio")
    end
end

Stock AVY is 16.049382716049383% of portfolio
Stock COO is 8.024691358024691% of portfolio
Stock T is 0.6172839506172839% of portfolio
Stock LUMN is 0.6172839506172839% of portfolio
Stock KDP is 0.6172839506172839% of portfolio
Stock SCHW is 5.555555555555555% of portfolio
Stock NVDA is 22.839506172839506% of portfolio
Stock ABBV is 1.2345679012345678% of portfolio
Stock EQIX is 25.308641975308642% of portfolio
Stock BR is 19.1358024691358% of portfolio


In [532]:
# print the sectors we buy
for i=1:stocks
    if x[i] > 0
        #loop through column 1 of s
        for j=1:50
            if s[j,1] == names(p)[i+1]
                for k=2:12
                    if s[j,k] == 1
                        println("Stock ", names(p)[i+1], " is in sector ", names(s)[k])
                    end
                end
            end
        end
    end
end

Stock AVY is in sector Materials
Stock COO is in sector Health Care
Stock T is in sector Communication Services
Stock LUMN is in sector Communication Services
Stock KDP is in sector Consumer Staples
Stock SCHW is in sector Financials
Stock NVDA is in sector Information Technology
Stock ABBV is in sector Health Care
Stock EQIX is in sector Real Estate
Stock BR is in sector Information Technology
